---
## 1️⃣ CONFIGURATION & API KEYS

In [1]:
# ═══════════════════════════════════════════════════════════════
# 🔑 API CONFIGURATION
# ═══════════════════════════════════════════════════════════════

import os

# Set API Keys
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY_HERE"

# Verify
print("✅ API Key configured!")
print(f"   GROQ_API_KEY: {os.environ['GROQ_API_KEY'][:20]}...")

✅ API Key configured!
   GROQ_API_KEY: YOUR_GROQ_API...


---
## 2️⃣ IMPORTS & DEPENDENCIES

In [3]:
# ═══════════════════════════════════════════════════════════════
# 📦 IMPORTS
# ═══════════════════════════════════════════════════════════════

# Core
from pathlib import Path
from datetime import datetime
import json
import tempfile

# LLM & AI
from groq import Groq
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Voice
from gtts import gTTS
from IPython.display import Audio, display

print("✅ All imports successful!")

ModuleNotFoundError: No module named 'langchain.text_splitter'

---
## 3️⃣ LLM MODEL INITIALIZATION

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🤖 LLM MODEL - Groq Llama 3.3 70B
# ═══════════════════════════════════════════════════════════════

# Initialize Groq client
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# LangChain ChatGroq for advanced usage
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=1500,
    groq_api_key=os.environ.get("GROQ_API_KEY")
)

# Test LLM
test_response = groq_client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Say 'LLM Ready' in 2 words"}],
    max_tokens=10
)
print(f"✅ LLM Model initialized!")
print(f"   Model: llama-3.3-70b-versatile")
print(f"   Test: {test_response.choices[0].message.content}")

---
## 4️⃣ EMBEDDINGS MODEL

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🧠 EMBEDDINGS - HuggingFace Sentence Transformers
# ═══════════════════════════════════════════════════════════════

# Initialize embeddings model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Test embeddings
test_embedding = embeddings.embed_query("Hello world")
print(f"✅ Embeddings model initialized!")
print(f"   Model: all-MiniLM-L6-v2")
print(f"   Embedding dimension: {len(test_embedding)}")

---
## 5️⃣ CHROMADB VECTOR DATABASE

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🗄️ CHROMADB - Vector Database for English Learning Content
# ═══════════════════════════════════════════════════════════════

CHROMA_DB_PATH = "english_learning_db"

def load_or_create_vectordb():
    """
    Load existing ChromaDB or create new one from data files.
    """
    # Check if DB exists
    if Path(CHROMA_DB_PATH).exists():
        print("📂 Loading existing ChromaDB...")
        vectordb = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embeddings
        )
        print(f"✅ ChromaDB loaded! Documents: {vectordb._collection.count()}")
        return vectordb
    
    # Create new DB from data files
    print("📚 Creating new ChromaDB from data files...")
    
    data_path = Path("data")
    all_texts = []
    all_metadatas = []
    
    # Load all .txt files from data folder
    if data_path.exists():
        for txt_file in data_path.glob("*.txt"):
            try:
                content = txt_file.read_text(encoding='utf-8')
                category = txt_file.stem  # filename without extension
                
                # Split into chunks
                text_splitter = RecursiveCharacterTextSplitter(
                    chunk_size=500,
                    chunk_overlap=50
                )
                chunks = text_splitter.split_text(content)
                
                for chunk in chunks:
                    all_texts.append(chunk)
                    all_metadatas.append({"category": category, "source": txt_file.name})
                
                print(f"   ✓ Loaded {txt_file.name}: {len(chunks)} chunks")
            except Exception as e:
                print(f"   ✗ Error loading {txt_file.name}: {e}")
    
    if all_texts:
        # Create ChromaDB
        vectordb = Chroma.from_texts(
            texts=all_texts,
            embedding=embeddings,
            metadatas=all_metadatas,
            persist_directory=CHROMA_DB_PATH
        )
        print(f"✅ ChromaDB created! Total documents: {len(all_texts)}")
        return vectordb
    else:
        print("⚠️ No data files found. Creating empty DB.")
        vectordb = Chroma(
            persist_directory=CHROMA_DB_PATH,
            embedding_function=embeddings
        )
        return vectordb

# Initialize ChromaDB
vectordb = load_or_create_vectordb()

---
## 6️⃣ RETRIEVAL FUNCTION

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🔍 RETRIEVAL - Search relevant content from ChromaDB
# ═══════════════════════════════════════════════════════════════

def retrieve_context(query, k=3):
    """
    Search ChromaDB for relevant English learning content.
    
    Args:
        query: User question
        k: Number of results to return
    
    Returns:
        Concatenated relevant content
    """
    try:
        results = vectordb.similarity_search(query, k=k)
        
        if results:
            context = "\n\n".join([doc.page_content for doc in results])
            return context
        return ""
    except Exception as e:
        print(f"Retrieval error: {e}")
        return ""

# Test retrieval
test_context = retrieve_context("present perfect tense")
print("✅ Retrieval function ready!")
print(f"   Test query: 'present perfect tense'")
print(f"   Retrieved: {len(test_context)} characters")

---
## 7️⃣ PROMPT TEMPLATES

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 📝 PROMPT TEMPLATES - Category-specific prompts
# ═══════════════════════════════════════════════════════════════

SYSTEM_PROMPT = """You are an expert English tutor. Your role is to help users learn English effectively.

User's level: {level}

When responding:
1. Be friendly, encouraging, and patient
2. Explain concepts clearly with examples
3. Correct grammar mistakes gently:
   📝 **Correction:** [what was wrong]
   ✅ **Better:** [corrected version]
   💡 **Tip:** [brief explanation]
4. Ask follow-up questions to encourage practice
5. Use the context provided when relevant

Context from knowledge base:
{context}
"""

CATEGORY_PROMPTS = {
    'grammar': """Focus on grammar rules. Explain:
1. The grammatical structure
2. When to use it
3. Common mistakes to avoid
4. 2-3 example sentences""",
    
    'vocabulary': """Focus on vocabulary. Include:
1. Clear definition
2. Pronunciation guide
3. Synonyms and antonyms
4. Example sentences in context""",
    
    'pronunciation': """Focus on pronunciation. Explain:
1. How to pronounce the sound/word
2. IPA transcription if relevant
3. Common pronunciation mistakes
4. Practice tips""",
    
    'conversation': """Focus on conversational English. Include:
1. Natural expressions to use
2. Common phrases
3. Cultural context if relevant
4. Practice dialogue examples""",
    
    'general': """Help the user with their English question.
Be comprehensive but concise."""
}

print("✅ Prompt templates defined!")
print(f"   Categories: {list(CATEGORY_PROMPTS.keys())}")

---
## 8️⃣ QUERY CLASSIFICATION

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🏷️ QUERY CLASSIFICATION - Detect question type
# ═══════════════════════════════════════════════════════════════

def classify_query(query):
    """
    Classify the user query into a learning category.
    
    Returns: category string
    """
    query_lower = query.lower()
    
    # Grammar keywords
    grammar_keywords = ['tense', 'verb', 'noun', 'adjective', 'adverb', 'preposition',
                        'article', 'pronoun', 'conditional', 'passive', 'active',
                        'present', 'past', 'future', 'perfect', 'continuous',
                        'grammar', 'structure', 'sentence', 'clause']
    
    # Vocabulary keywords
    vocab_keywords = ['word', 'meaning', 'vocabulary', 'synonym', 'antonym',
                      'definition', 'means', 'mean', 'what is', "what's"]
    
    # Pronunciation keywords
    pronunciation_keywords = ['pronounce', 'pronunciation', 'sound', 'accent',
                              'speak', 'say', 'ipa', 'phonetic']
    
    # Conversation keywords
    conversation_keywords = ['conversation', 'dialogue', 'talk', 'speak',
                             'expression', 'phrase', 'idiom', 'slang']
    
    # Check categories
    if any(kw in query_lower for kw in grammar_keywords):
        return 'grammar'
    elif any(kw in query_lower for kw in vocab_keywords):
        return 'vocabulary'
    elif any(kw in query_lower for kw in pronunciation_keywords):
        return 'pronunciation'
    elif any(kw in query_lower for kw in conversation_keywords):
        return 'conversation'
    else:
        return 'general'

# Test classification
test_queries = [
    "What is the present perfect tense?",
    "What does 'serendipity' mean?",
    "How do I pronounce 'thorough'?",
    "How to start a conversation?"
]

print("✅ Query classifier ready!")
for q in test_queries:
    print(f"   '{q[:30]}...' → {classify_query(q)}")

---
## 9️⃣ CHAT ENGINE

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 💬 CHAT ENGINE - Main conversation handler
# ═══════════════════════════════════════════════════════════════

# Conversation history
conversation_history = []

# User settings
user_settings = {
    "level": "Intermediate",  # Beginner, Intermediate, Advanced
    "voice_enabled": True,
    "correction_enabled": True
}

def chat(user_message, use_rag=True):
    """
    Main chat function. Processes user message and returns AI response.
    
    Args:
        user_message: User's question or message
        use_rag: Whether to use RAG (retrieve from ChromaDB)
    
    Returns:
        AI response string
    """
    global conversation_history
    
    # Classify query
    category = classify_query(user_message)
    
    # Retrieve context if RAG enabled
    context = ""
    if use_rag:
        context = retrieve_context(user_message, k=3)
    
    # Build system prompt
    system_prompt = SYSTEM_PROMPT.format(
        level=user_settings["level"],
        context=context if context else "No specific context available."
    )
    
    # Add category-specific instructions
    category_instruction = CATEGORY_PROMPTS.get(category, CATEGORY_PROMPTS['general'])
    system_prompt += f"\n\n{category_instruction}"
    
    # Add to history
    conversation_history.append({"role": "user", "content": user_message})
    
    try:
        # Call LLM
        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": system_prompt},
                *conversation_history[-10:]  # Keep last 10 messages
            ],
            max_tokens=1000,
            temperature=0.7
        )
        
        assistant_message = response.choices[0].message.content
        
        # Add to history
        conversation_history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
        
    except Exception as e:
        error_msg = f"Error: {type(e).__name__}: {e}"
        return error_msg

def clear_history():
    """Clear conversation history"""
    global conversation_history
    conversation_history = []
    print("✅ Conversation history cleared!")

def set_level(level):
    """Set user's English level"""
    user_settings["level"] = level
    print(f"✅ Level set to: {level}")

print("✅ Chat engine ready!")
print(f"   Level: {user_settings['level']}")
print(f"   Voice: {user_settings['voice_enabled']}")

---
## 🔟 TEXT-TO-SPEECH (TTS)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🔊 TEXT-TO-SPEECH - Voice output
# ═══════════════════════════════════════════════════════════════

def speak(text, accent="com"):
    """
    Convert text to speech and play it.
    
    Args:
        text: Text to speak
        accent: 'com' (US), 'co.uk' (UK), 'com.au' (Australian)
    """
    try:
        # Generate speech
        tts = gTTS(text=text[:500], lang='en', tld=accent)
        
        # Save to temp file
        audio_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        tts.save(audio_file.name)
        
        # Display audio player in notebook
        display(Audio(audio_file.name, autoplay=True))
        
        return audio_file.name
    except Exception as e:
        print(f"TTS Error: {e}")
        return None

print("✅ Text-to-Speech ready!")
print("   Usage: speak('Hello world')")

---
## 1️⃣1️⃣ CHATBOT CLASS (For Streamlit Export)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🤖 CHATBOT CLASS - Complete encapsulated chatbot
# ═══════════════════════════════════════════════════════════════

class EnglishLearningChatbot:
    """
    Complete English Learning Chatbot class.
    Can be imported into Streamlit app.
    """
    
    def __init__(self, api_key=None):
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        self.client = Groq(api_key=self.api_key)
        self.history = []
        self.level = "Intermediate"
        self.vectordb = vectordb
        
    def chat(self, message):
        """Send message and get response"""
        return chat(message)
    
    def speak(self, text):
        """Convert text to speech"""
        return speak(text)
    
    def clear(self):
        """Clear history"""
        clear_history()
    
    def set_level(self, level):
        """Set English level"""
        set_level(level)
        self.level = level

# Create global instance
chatbot = EnglishLearningChatbot()

print("✅ EnglishLearningChatbot class ready!")
print("   Usage: chatbot.chat('your question')")

---
# 🎯 INTERACTIVE CHAT
## Use the cells below to chat with the bot!

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 💬 CHAT - Change the question and run this cell!
# ═══════════════════════════════════════════════════════════════

# 👇 Change your question here:
question = "What is the conditional tense in English?"

# Get response
print("="*60)
print(f"👤 You: {question}")
print("="*60)

response = chat(question)

print(f"\n🤖 Tutor:\n{response}")
print("\n" + "="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════
# 🔊 LISTEN TO RESPONSE - Run to hear the last response
# ═══════════════════════════════════════════════════════════════

if conversation_history:
    last_response = conversation_history[-1]["content"]
    print("🔊 Playing audio...")
    speak(last_response)
else:
    print("⚠️ No response to play. Chat first!")

In [ ]:
# ═══════════════════════════════════════════════════════════════
# ⚙️ SETTINGS - Change chatbot settings
# ═══════════════════════════════════════════════════════════════

# Set your English level:
# Options: "Beginner", "Intermediate", "Advanced"
set_level("Intermediate")

# Clear conversation history (uncomment to use):
# clear_history()

---
## 📤 EXPORT FOR STREAMLIT

The `EnglishLearningChatbot` class can be imported into your Streamlit app:

```python
# In your streamlit app:
from ENGLISH_CHATBOT_IMPLEMENTATION import chatbot

response = chatbot.chat("What is present perfect?")
chatbot.speak(response)
```